In [108]:
import datetime as dt 
import pandas as pd
import yfinance as yf 
import math
import numpy as np
from scipy.optimize import minimize
from pandas.tseries.offsets import BDay

#my current holdings 
NVDA = yf.Ticker('NVDA')
BTCC = yf.Ticker('BTCC-B.TO')
ENB = yf.Ticker('ENB.TO')
SIA = yf.Ticker('SIA.TO')
XSP = yf.Ticker('XSP.TO')
EXrate = yf.Ticker('CAD=X')

#holdings current price
current_NVDA = NVDA.fast_info['lastPrice']
current_BTCC = BTCC.fast_info['lastPrice']
current_ENB = ENB.fast_info['lastPrice']
current_SIA = SIA.fast_info['lastPrice']
current_XSP = XSP.fast_info['lastPrice']
current_EXrate = EXrate.fast_info['lastPrice']

#avg price bought 
NVDA_avg = 63.
BTCC_avg = 9.6899
ENB_avg = 45.2595
SIA_avg = 9.9163
XSP_avg = 35.3838

#shares each 
NVDA_shares = 100
BTCC_shares = 500
ENB_shares = 100
SIA_shares = 300
XSP_shares = 100

#total
cash = 40104

equity = (NVDA_shares*(NVDA_avg+current_NVDA-NVDA_avg))*current_EXrate + BTCC_shares*(BTCC_avg+current_BTCC-BTCC_avg) + ENB_shares*(ENB_avg+current_ENB-ENB_avg) + SIA_shares*(SIA_avg+current_SIA-SIA_avg) + XSP_shares*(XSP_avg+current_XSP-XSP_avg)

account_balance = cash+equity
print(account_balance)



78129.14828354589


In [109]:
def volatility(Ticker):
    #monthly adjusted close prices for NVDA
    df = yf.download(Ticker, period = 'max', interval='1mo', auto_adjust=True)

    #monthly returns
    sum = 0
    months = 0
    monthlyReturns = []
    
    for i in range(1, len(df['Close'])):
        monthlyReturn = math.log(df.iloc[i]['Close'] / df.iloc[i-1]['Close'])
        sum += monthlyReturn
        monthlyReturns.append(monthlyReturn)
        months += 1

    # average monthly return
    monthlyReturnAvg = sum/(months)

    #variance
    squareSum = 0
    for r in monthlyReturns:    
        deviation = r - monthlyReturnAvg
        squareSum += deviation ** 2

    variance = squareSum / (len(monthlyReturns))

    #volatility
    volatility = math.sqrt(variance)

    #annualized
    print(volatility)

volatility('NVDA')
volatility('BTCC-B.TO')
volatility('ENB.TO')
volatility('SIA.TO')
volatility('XSP.TO')





[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


0.170912670236881
0.18746773319396562
0.05069080732361892


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

0.05879674474501474
0.048289258401307635


In [164]:


#returns + cov_matrix
def returns_cov(tickers, start_date, end_date):
    data = yf.download(tickers, start = start_date, end = end_date)['Adj Close']
    returns = data.pct_change().dropna()
    cov_matrix = returns.cov()
    return returns, cov_matrix

#portfolio variance
def variance(w, cov_matrix):
    variance = np.dot(w.T, np.dot(cov_matrix, w))
    return variance

#risk_contribution
def rc(w, cov_matrix):
    port_var = variance(w, cov_matrix)
    mc = np.dot(cov_matrix, w)
    rc = np.multiply(mc, w)/port_var
    return rc

#objective function
def objective(w, cov_matrix):
    n = len(w)
    target_risk = 1.0/n
    risk = rc(w, cov_matrix)
    return np.sum((risk - target_risk) ** 2)

#contraint that weights sum to 1
def constraint_sum(w):
    return np.sum(w)-1


def erc(tickers, start_date, end_date):
    returns, cov_matrix = returns_cov(tickers, start_date, end_date)
    initial_weights = np.ones(len(tickers)) / len(tickers)
    bounds = [(0,1) for i in range(len(tickers))]
    constraints  = ({'type': 'eq', 'fun': constraint_sum})

    result = minimize(objective, initial_weights, args=(cov_matrix), method = 'SLSQP', bounds=bounds, constraints = constraints)

    return result.x


tickers = ['NVDA','BTCC-B.TO', 'ENB.TO', 'SIA.TO', 'XSP.TO']
end_date = dt.datetime.now()
start_date = end_date - BDay(365)



weights = erc(tickers, start_date, end_date)

print("Equal Risk Contribution Portfolio Weights:")
print(weights)


[*********************100%%**********************]  5 of 5 completed

Equal Risk Contribution Portfolio Weights:
[0.09875399 0.30255451 0.09931548 0.20758163 0.29179439]



/var/folders/85/j4fbqr9109s00vvxrh0bj9nm0000gp/T/ipykernel_64926/649327384.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()
